In [1]:
from __future__ import print_function

from darkflow.net.build import TFNet

import argparse
import socket
import sys
import binascii
import struct
from collections import namedtuple
import cv2
import numpy as np
import matplotlib.pyplot as plt

W0620 15:44:31.175668 17404 deprecation_wrapper.py:119] From c:\Users\BME-4\Anaconda3\envs\cv\lib\site-packages\darkflow\net\build.py:15: The name tf.train.RMSPropOptimizer is deprecated. Please use tf.compat.v1.train.RMSPropOptimizer instead.

W0620 15:44:31.183669 17404 deprecation_wrapper.py:119] From c:\Users\BME-4\Anaconda3\envs\cv\lib\site-packages\darkflow\net\build.py:16: The name tf.train.AdadeltaOptimizer is deprecated. Please use tf.compat.v1.train.AdadeltaOptimizer instead.

W0620 15:44:31.184668 17404 deprecation_wrapper.py:119] From c:\Users\BME-4\Anaconda3\envs\cv\lib\site-packages\darkflow\net\build.py:17: The name tf.train.AdagradOptimizer is deprecated. Please use tf.compat.v1.train.AdagradOptimizer instead.

W0620 15:44:31.185668 17404 deprecation_wrapper.py:119] From c:\Users\BME-4\Anaconda3\envs\cv\lib\site-packages\darkflow\net\build.py:18: The name tf.train.AdagradDAOptimizer is deprecated. Please use tf.compat.v1.train.AdagradDAOptimizer instead.

W0620 15:44:31

In [15]:
import sys

In [2]:
options = {"model": "cfg/yolo.cfg", 
           "load": "bin/yolo.weights", 
           "threshold": 0.1, 
           "gpu": 1.0}

tfnet = TFNet(options)

W0620 15:44:41.951033 17404 deprecation_wrapper.py:119] From c:\Users\BME-4\Anaconda3\envs\cv\lib\site-packages\darkflow\net\build.py:105: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0620 15:44:41.954032 17404 deprecation_wrapper.py:119] From c:\Users\BME-4\Anaconda3\envs\cv\lib\site-packages\darkflow\net\ops\baseop.py:70: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0620 15:44:41.956040 17404 deprecation_wrapper.py:119] From c:\Users\BME-4\Anaconda3\envs\cv\lib\site-packages\darkflow\net\ops\baseop.py:71: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0620 15:44:41.968029 17404 deprecation_wrapper.py:119] From c:\Users\BME-4\Anaconda3\envs\cv\lib\site-packages\darkflow\net\ops\baseop.py:84: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0620 15:44:42.016028 17404 deprecation_wrapper.py:119] Fr

Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.02600550651550293s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep!

W0620 15:44:45.974028 17404 deprecation.py:506] From c:\Users\BME-4\Anaconda3\envs\cv\lib\site-packages\darkflow\net\ops\convolution.py:28: calling extract_image_patches (from tensorflow.python.ops.array_ops) with ksizes is deprecated and will be removed in a future version.
Instructions for updating:
ksizes is deprecated, use sizes instead


 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 19, 19, 1024)
 Load  |  Yep!  | concat [16]                      | (?, 38, 38, 512)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 38, 38, 64)
 Load  |  Yep!  | local flatten 2x2                | (?, 19, 19, 256)
 Load  |  Yep!  | concat [27, 24]                  | (?, 19, 19, 1280)


W0620 15:44:46.920028 17404 deprecation_wrapper.py:119] From c:\Users\BME-4\Anaconda3\envs\cv\lib\site-packages\darkflow\net\build.py:132: The name tf.GPUOptions is deprecated. Please use tf.compat.v1.GPUOptions instead.



 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 19, 19, 1024)
 Load  |  Yep!  | conv 1x1p0_1    linear           | (?, 19, 19, 425)
-------+--------+----------------------------------+---------------
GPU mode with 1.0 usage
Finished in 15.463421821594238s



In [3]:
def boxing(original_img, predictions):
    newImage = np.copy(original_img)

    for result in predictions:
        top_x = result['topleft']['x']
        top_y = result['topleft']['y']

        btm_x = result['bottomright']['x']
        btm_y = result['bottomright']['y']

        confidence = result['confidence']
        label = result['label'] + " " + str(round(confidence, 3))

        if confidence > 0.3:
            newImage = cv2.rectangle(newImage, (top_x, top_y), (btm_x, btm_y), (255,0,0), 3)
            newImage = cv2.putText(newImage, label, (top_x, top_y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL , 0.8, (0, 230, 0), 1, cv2.LINE_AA)
            
    return newImage

#_, ax = plt.subplots(figsize=(20, 10))

In [6]:
#fourcc = cv2.VideoWriter_fourcc(*'DIVX')

#out = cv2.VideoWriter('./sample_video/output.avi',fourcc, 20.0, (int(width), int(height)))

img1=cv2.imread('gun.jpg')

print(np.asarray(img1).shape)
#img2=cv2.resize(img1, (152,152))
img=np.asarray(img1)
print(img.shape)
results = tfnet.return_predict(img)

new_frame = boxing(img, results)
print(new_frame.shape)
#cv2.imshow('dwd',new_frame)
#out.write(new_frame)

#cv2.imshow('frame',new_frame)
#out.write(img)
cv2.imshow('frame',new_frame)
cv2.waitKey(0); 
cv2.destroyAllWindows(); 
cv2.waitKey(1)
results

(406, 700, 3)
(406, 700, 3)
(406, 700, 3)


[{'label': 'person',
  'confidence': 0.10301071,
  'topleft': {'x': 307, 'y': 43},
  'bottomright': {'x': 325, 'y': 71}},
 {'label': 'person',
  'confidence': 0.44894347,
  'topleft': {'x': 372, 'y': 3},
  'bottomright': {'x': 423, 'y': 116}},
 {'label': 'person',
  'confidence': 0.29144457,
  'topleft': {'x': 407, 'y': 4},
  'bottomright': {'x': 441, 'y': 103}},
 {'label': 'person',
  'confidence': 0.17737837,
  'topleft': {'x': 147, 'y': 2},
  'bottomright': {'x': 237, 'y': 151}},
 {'label': 'person',
  'confidence': 0.12745857,
  'topleft': {'x': 228, 'y': 48},
  'bottomright': {'x': 264, 'y': 110}},
 {'label': 'person',
  'confidence': 0.6459309,
  'topleft': {'x': 244, 'y': 46},
  'bottomright': {'x': 280, 'y': 112}},
 {'label': 'person',
  'confidence': 0.19171444,
  'topleft': {'x': 327, 'y': 44},
  'bottomright': {'x': 354, 'y': 109}},
 {'label': 'person',
  'confidence': 0.12657483,
  'topleft': {'x': 366, 'y': 32},
  'bottomright': {'x': 400, 'y': 113}},
 {'label': 'person',


In [ ]:
PROCESS = True

# Definitions

# Protocol Header Format
# Cookie VersionMajor VersionMinor FrameType Timestamp ImageWidth
# ImageHeight PixelStride RowStride
SENSOR_STREAM_HEADER_FORMAT = "@IBBHqIIII"

SENSOR_FRAME_STREAM_HEADER = namedtuple(
    'SensorFrameStreamHeader',
    'Cookie VersionMajor VersionMinor FrameType Timestamp ImageWidth ImageHeight PixelStride RowStride'
)

# Each port corresponds to a single stream type
# Port for obtaining Photo Video Camera stream
PV_STREAM_PORT = 23940


def main(argv):
    """Receiver main"""
    parser = argparse.ArgumentParser()
    required_named_group = parser.add_argument_group('named arguments')

    required_named_group.add_argument("-a", "--host",
                                      help="Host address to connect", required=True)
    args = parser.parse_args(argv)

    # Create a TCP Stream socket
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    except (socket.error, msg):
        print("ERROR: Failed to create socket. Code: " + str(msg[0]) + ', Message: ' + msg[1])
        sys.exit()

    print('INFO: socket created')

    # Try connecting to the address
    s.connect((args.host, PV_STREAM_PORT))

    print('INFO: Socket Connected to ' + args.host + ' on port ' + str(PV_STREAM_PORT))

    # Try receive data
    try:
        quit = False
        while not quit:
            reply = s.recv(struct.calcsize(SENSOR_STREAM_HEADER_FORMAT))
            if not reply:
                print('ERROR: Failed to receive data')
                sys.exit()

            data = struct.unpack(SENSOR_STREAM_HEADER_FORMAT, reply)

            # Parse the header
            header = SENSOR_FRAME_STREAM_HEADER(*data)

            # read the image in chunks
            image_size_bytes = header.ImageHeight * header.RowStride
            image_data = ''

            while len(image_data) < image_size_bytes:
                remaining_bytes = image_size_bytes - len(image_data)
                image_data_chunk = s.recv(remaining_bytes)
                if not image_data_chunk:
                    print('ERROR: Failed to receive image data')
                    sys.exit()
                image_data += image_data_chunk

            image_array = np.frombuffer(image_data, dtype=np.uint8).reshape((header.ImageHeight,
                                        header.ImageWidth, header.PixelStride))
            if PROCESS:
                # process image
                #gray = cv2.cvtColor(image_array,cv2.COLOR_BGR2GRAY)
                #image_array = cv2.Canny(gray,50,150,apertureSize = 3)
                results = tfnet.return_predict(image_array)
                new_frame = boxing(img, results)
                #cv2.imshow('frame',new_frame)
                #cv2.waitKey(0)
            
            
            
            cv2.imshow('Photo Video Camera Stream', new_frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    except KeyboardInterrupt:
        pass

    s.close()
    cv2.destroyAllWindows()
    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main(sys.argv[1:])
    #main(192.168.100)

In [ ]:
from __future__ import print_function
import darkflow
from darkflow.net.build import TFNet

import argparse
import socket
import sys
import binascii
import struct
from collections import namedtuple
import cv2
import numpy as np
import matplotlib.pyplot as plt

options = {"model": "cfg/yolo.cfg", 
           "load": "bin/yolo.weights", 
           "threshold": 0.1, 
           "gpu": 1.0}

tfnet = TFNet(options)
def boxing(original_img, predictions):
    newImage = np.copy(original_img)

    for result in predictions:
        top_x = result['topleft']['x']
        top_y = result['topleft']['y']

        btm_x = result['bottomright']['x']
        btm_y = result['bottomright']['y']

        confidence = result['confidence']
        label = result['label'] + " " + str(round(confidence, 3))

        if confidence > 0.3:
            newImage = cv2.rectangle(newImage, (top_x, top_y), (btm_x, btm_y), (255,0,0), 3)
            newImage = cv2.putText(newImage, label, (top_x, top_y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL , 0.8, (0, 230, 0), 1, cv2.LINE_AA)
            
    return newImage

PROCESS = True

# Definitions

# Protocol Header Format
# Cookie VersionMajor VersionMinor FrameType Timestamp ImageWidth
# ImageHeight PixelStride RowStride
SENSOR_STREAM_HEADER_FORMAT = "@IBBHqIIII"

SENSOR_FRAME_STREAM_HEADER = namedtuple(
    'SensorFrameStreamHeader',
    'Cookie VersionMajor VersionMinor FrameType Timestamp ImageWidth ImageHeight PixelStride RowStride'
)

# Each port corresponds to a single stream type
# Port for obtaining Photo Video Camera stream
PV_STREAM_PORT = 23940


def main(argv):
    """Receiver main"""
    parser = argparse.ArgumentParser()
    required_named_group = parser.add_argument_group('named arguments')

    required_named_group.add_argument("-a", "--host",
                                      help="Host address to connect", required=True)
    args = parser.parse_args(argv)

    # Create a TCP Stream socket
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    except (socket.error, msg):
        print("ERROR: Failed to create socket. Code: " + str(msg[0]) + ', Message: ' + msg[1])
        sys.exit()

    print('INFO: socket created')

    # Try connecting to the address
    s.connect((args.host, PV_STREAM_PORT))

    print('INFO: Socket Connected to ' + args.host + ' on port ' + str(PV_STREAM_PORT))

    # Try receive data
    try:
        quit = False
        while not quit:
            reply = s.recv(struct.calcsize(SENSOR_STREAM_HEADER_FORMAT))
            if not reply:
                print('ERROR: Failed to receive data')
                sys.exit()

            data = struct.unpack(SENSOR_STREAM_HEADER_FORMAT, reply)

            # Parse the header
            header = SENSOR_FRAME_STREAM_HEADER(*data)

            # read the image in chunks
            image_size_bytes = header.ImageHeight * header.RowStride
            image_data = bytes()
            print("Data Receiver ______________")
            while len(image_data) < image_size_bytes:
                remaining_bytes = image_size_bytes - len(image_data)
                image_data_chunk = s.recv(remaining_bytes)
                if not image_data_chunk:
                    print('ERROR: Failed to receive image data')
                    sys.exit()
                image_data += image_data_chunk

            image_array = np.frombuffer(image_data, dtype=np.uint8).reshape((header.ImageHeight,
                                        header.ImageWidth, header.PixelStride))
            if PROCESS:
                # process image
                #gray = cv2.cvtColor(image_array,cv2.COLOR_BGR2GRAY)
                #image_array = cv2.Canny(gray,50,150,apertureSize = 3)
                image_array=image_array[:,:,:3]
                image_array=cv2.resize(image_array, (320,180))
                print("going to resuklts",image_array.shape)
                results = tfnet.return_predict(image_array[:,:,:3])
                new_frame = boxing(image_array[:,:,:3], results)
                #cv2.imshow('frame',new_frame)
                #cv2.waitKey(0)
            
            
            
            cv2.imshow('Photo Video Camera Stream', new_frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    except KeyboardInterrupt:
        pass

    s.close()
    cv2.destroyAllWindows()
    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main(sys.argv[1:])
    #main(192.168.100)